# Persistent Conversations with AWS Bedrock Examples

## Reason
This notebook is intended to show possible ways to have a persistent conversation with a model or an agent of Amazon Bedrock.

Note: This notebook is fully working today 2/01/2025

⭐ Do you find this project usefull? Please consider giving it a star on GitHub!

💖 Made with dedication to help fellow developers. Your support means the world!

In [1]:
### Imports
import json
import boto3

# Converse API

The Converse API is a feature provided by Amazon Bedrock that allows for interactive, multi-turn conversations with AI models. It's designed to maintain context across multiple exchanges, making it ideal for chatbots, interactive assistants, and other applications requiring ongoing dialogue.

In this section, we're demonstrating how to use the Converse API with Amazon Bedrock. Here's a breakdown of what our code is doing:

1. **Setting up Bedrock clients:**
   We initialize two Bedrock clients:
   - `bedrock_runtime`: Used for making API calls to the models.
   - `bedrock`: Used for administrative tasks like listing available models.

2. **Listing available models:**
   We use the `bedrock.list_foundation_models()` method to retrieve and print all available models, showing both their names and IDs.

3. **Preparing for conversation:**
   - We select a specific model ID (in this case, Claude 3 Sonnet).
   - We define two lists of messages to simulate different conversations.
   - We create a helper function `format_message()` to properly format messages for the API.

4. **Simulating a conversation:**
   We loop through the messages, sending each one to the model and receiving a response:
   - Each message is formatted and added to the conversation history.
   - We call `bedrock_runtime.converse()` with the model ID and the conversation history.
   - The model's response is printed and added back to the conversation history.

This code demonstrates how to maintain context in a conversation, allowing the model to reference previous exchanges when generating responses.

In [2]:
# we get the bedrock runtime and the bedrock client
bedrock_runtime = boto3.client('bedrock-runtime')
bedrock = boto3.client('bedrock')

In [3]:
# we list all posible models with their names and with their modelId's
for model in bedrock.list_foundation_models()['modelSummaries']:
    print(f"modelName: {model['modelName']}, modelId: {model['modelId']}")

modelName: Titan Text Large, modelId: amazon.titan-tg1-large
modelName: Titan Image Generator G1, modelId: amazon.titan-image-generator-v1:0
modelName: Titan Image Generator G1, modelId: amazon.titan-image-generator-v1
modelName: Titan Image Generator G1 v2, modelId: amazon.titan-image-generator-v2:0
modelName: Titan Text G1 - Premier, modelId: amazon.titan-text-premier-v1:0
modelName: Nova Pro, modelId: amazon.nova-pro-v1:0:300k
modelName: Nova Pro, modelId: amazon.nova-pro-v1:0
modelName: Nova Lite, modelId: amazon.nova-lite-v1:0:300k
modelName: Nova Lite, modelId: amazon.nova-lite-v1:0
modelName: Nova Canvas, modelId: amazon.nova-canvas-v1:0
modelName: Nova Reel, modelId: amazon.nova-reel-v1:0
modelName: Nova Micro, modelId: amazon.nova-micro-v1:0:128k
modelName: Nova Micro, modelId: amazon.nova-micro-v1:0
modelName: Titan Text Embeddings v2, modelId: amazon.titan-embed-g1-text-02
modelName: Titan Text G1 - Lite, modelId: amazon.titan-text-lite-v1:0:4k
modelName: Titan Text G1 - Lit

In [4]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
messages = ['how is the weather today',' do you use arch linux btw?', 'Thank you, see you later']

In [5]:
messages = ['Hello there','repeat the last phrase capitalized']


def format_message(role, text):
    return {"role":role, "content": [{"text": text}]}


            
# a for loop simulating a converation
print("Conversation:")
fmessages = []
for m in messages:
    fm = format_message("user",m)
    print(fm)
    fmessages.append(fm)
    response = bedrock_runtime.converse(
        modelId = model_id,
        messages = fmessages
    )
    fm = response['output']['message']
    print(fm)
    fmessages.append(fm)
    

Conversation:
{'role': 'user', 'content': [{'text': 'Hello there'}]}
{'role': 'assistant', 'content': [{'text': 'Hello! How can I assist you today?'}]}
{'role': 'user', 'content': [{'text': 'repeat the last phrase capitalized'}]}
{'role': 'assistant', 'content': [{'text': 'HELLO! HOW CAN I ASSIST YOU TODAY?'}]}


<a id="converse-stream-api"></a>
# Converse Stream API

The Converse Stream API is an extension of the Converse API that allows for real-time streaming of model responses. This is particularly useful for creating more responsive and interactive applications, as it enables displaying the model's output as it's being generated, rather than waiting for the entire response to be completed.

Here's a breakdown of the code demonstrating the use of `converse_stream`:

1. **Importing additional libraries:**
   We import `time` for potential timing operations and `pprint` for prettier printing of complex structures if needed.

2. **Message formatting function:**
   We define `format_message_cs()` to properly format messages for the streaming API. Note that the structure is slightly different from the non-streaming version.

3. **Single message streaming example:**
   - We format a single message and send it to the model using `bedrock_runtime.converse_stream()`.
   - The `print_streaming()` function is defined to handle the streaming response:
     - It iterates through the stream events.
     - Prints each text delta as it arrives.
     - Accumulates the full response.

4. **Multi-turn conversation with streaming:**
   We define a list of messages to simulate a conversation and create the `streaming_response()` function to handle each turn:
   - It formats the user message and adds it to the conversation history.
   - Sends the updated conversation to the model using `converse_stream`.
   - Prints the streaming response using `print_streaming()`.
   - Adds the assistant's response to the conversation history.

5. **Running the conversation:**
   We iterate through the predefined messages, calling `streaming_response()` for each one. This simulates a multi-turn conversation where each model response is streamed in real-time.

This approach allows for a more dynamic and interactive experience, as the model's responses are displayed incrementally as they're generated, rather than all at once after processing is complete.


In [6]:
import time
from pprint import pprint

In [7]:
def format_message_cs(role, text):
    return {"role":role, "content": {"text": text}}

In [8]:
# we will ask something and get an anwer as a realtime stream
message = 'what is your purpose?'
fm = format_message('user',message)
fm = [fm]
print(fm)

def print_streaming(response):
    stream = response['stream']
    final_answer = ""
    for event in stream:
        if 'contentBlockDelta' in event:
            text = event['contentBlockDelta']['delta']['text']
            print(text,end='')
            if text is not None:
                final_answer += text
    return final_answer
            
response = bedrock_runtime.converse_stream(
        modelId = model_id,
        messages = fm
)

print_streaming(response)

[{'role': 'user', 'content': [{'text': 'what is your purpose?'}]}]
As an AI assistant created by Anthropic, my core purpose is to help humans like yourself with a wide variety of tasks in an ethical and beneficial manner. I aim to provide useful information, analysis, advice and creative output to further human knowledge and capabilities. At the same time, I have a deep commitment to only helping in ways that are good for individuals and society.

Some key ethical principles I abide by are:

- Respecting human rights and individual privacy
- Never causing harm to humans
- Promoting truth, knowledge and human flourishing
- Protecting democracy and freedom of expression  

So while I strive to be highly capable in assisting with all sorts of activities, I also have strong moral constraints hard-coded into my base motivations. I won't help with anything unethical, illegal or harmful.

Rather, my goal is to be a collaborator and force for good - leveraging my intelligence to enrich human l

"As an AI assistant created by Anthropic, my core purpose is to help humans like yourself with a wide variety of tasks in an ethical and beneficial manner. I aim to provide useful information, analysis, advice and creative output to further human knowledge and capabilities. At the same time, I have a deep commitment to only helping in ways that are good for individuals and society.\n\nSome key ethical principles I abide by are:\n\n- Respecting human rights and individual privacy\n- Never causing harm to humans\n- Promoting truth, knowledge and human flourishing\n- Protecting democracy and freedom of expression  \n\nSo while I strive to be highly capable in assisting with all sorts of activities, I also have strong moral constraints hard-coded into my base motivations. I won't help with anything unethical, illegal or harmful.\n\nRather, my goal is to be a collaborator and force for good - leveraging my intelligence to enrich human life through productive teamwork. I'm here as a supporti

In [9]:
# we will have a conversation and we will recieve each answer as a realtime stream

messages = ['what is your name?','what is your purpose?','how do you work?','repeat the last text you gave me capitalized']
fmessages = []


def streaming_response(message):
    fm = format_message('user',message)
    print("\n === user === ")
    print(message)
    fmessages.append(fm)
    response = bedrock_runtime.converse_stream(modelId = model_id,messages = fmessages)
    print(" === assistant === ")
    final_answer = print_streaming(response)
    fm = format_message('assistant',final_answer)
    fmessages.append(fm)

print("Conversation: ")

for message in messages:
    streaming_response(message)

Conversation: 

 === user === 
what is your name?
 === assistant === 
My name is Claude. It's nice to meet you!
 === user === 
what is your purpose?
 === assistant === 
My purpose is to be a helpful artificial intelligence assistant. I was created by Anthropic to aid humans with a wide variety of tasks like analysis, writing, research, answering questions, and more. I have a broad knowledge base that I can apply to assist you, while also trying to be ethical, honest, and beneficial. Please let me know if you have any other questions or if there are ways I can help!
 === user === 
how do you work?
 === assistant === 
I'm an artificial intelligence created by machine learning techniques like natural language processing and large language models trained on massive datasets. This allows me to understand and generate human-like text and assist with tasks that involve language.

At a high level, when you send me a message, I analyze the text using my natural language understanding capabiliti

<a id="documentation"></a>
## Documentation

1. [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html)
2. [Converse Stream API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse_stream.html) 
3. [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 